In [18]:
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load data and preprocess abit

In [19]:
train_data = pd.read_csv("train AG_news.csv")
test_data = pd.read_csv("test AG_news.csv")

In [20]:
train_data.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [21]:
print(f"{train_data.shape=}\n{test_data.shape=}") 

train_data.shape=(120000, 3)
test_data.shape=(7600, 3)


In [22]:
train_data["full_text"] = train_data.Title + " " + train_data.Description
train_data.drop(columns= ["Title", "Description"], inplace= True)
test_data["full_text"] = test_data.Title + " " + test_data.Description
test_data.drop(columns= ["Title", "Description"], inplace= True)
train_data.head()

,Class Index,full_text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."
